# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report

# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /Users/michael/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/michael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/michael/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse.db', engine)
X = df.message.values
Y = df.iloc[:, 4:].values

In [41]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV

# # Split data into features (X) and target labels (y)
# X = df['message']
# y = df[['related', 'request', 'offer',
#        'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
#        'security', 'military', 'child_alone', 'water', 'food', 'shelter',
#        'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
#        'infrastructure_related', 'transport', 'buildings', 'electricity',
#        'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
#        'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
#        'other_weather', 'direct_report']]

# # Define a pipeline with TfidfVectorizer and MultiOutputClassifier(SVC)
# pipeline = Pipeline([
#     ('vectorizer', TfidfVectorizer(tokenizer=word_tokenize)),
#     ('classifier', MultiOutputClassifier(SVC()))
# ])

# # Define the hyperparameter grid for SVC
# param_grid = {
#     'classifier__estimator__C': [0.1, 1, 10, 100],
#     'classifier__estimator__kernel': ['linear', 'rbf']
# }

# # Create a grid search object using the pipeline and parameter grid
# grid_search = GridSearchCV(pipeline, param_grid, cv=5)

# # Fit the grid search to the data
# grid_search.fit(X, y)

# # Print the best hyperparameters found by grid search
# print("Best hyperparameters:", grid_search.best_params_)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    stop_words = stopwords.words("english") + list(string.punctuation)
    lemmatizer = WordNetLemmatizer()
    
    text = text.lower().strip()
    tokens = word_tokenize(text)
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)
pipeline.fit(X_train, Y_train)

/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fd07a8416c0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)

In [10]:
list_precision, list_recall, list_f1 = [], [], []

# Iterate 36 target columns and generate a classification report for each
for i, col in enumerate(df.columns[4:]):
    
    rslt = classification_report(Y_test[:, i], Y_pred[:, i])
    
    # weighted avg scores are in the 2nd last line: 
    score_line = rslt.split('\n')[-2]
    score_line_split = score_line.split()
    
    # scores are in the 2nd to 4th places of splitted texts of score_line
    precision_score = float (score_line_split[2])
    list_precision.append(precision_score)
    
    recall_score = float (score_line_split[3])
    list_recall.append(recall_score)
    
    f1_score = float (score_line_split[4])
    list_f1.append(f1_score)
    
    print(f'{i} Target column {col}:')
    print(rslt)
    print()

0 Target column related:
              precision    recall  f1-score   support

           0       0.69      0.40      0.51      1509
           1       0.84      0.95      0.89      5004
           2       0.88      0.17      0.29        41

    accuracy                           0.82      6554
   macro avg       0.80      0.51      0.56      6554
weighted avg       0.80      0.82      0.80      6554


1 Target column request:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5419
           1       0.84      0.47      0.60      1135

    accuracy                           0.89      6554
   macro avg       0.87      0.72      0.77      6554
weighted avg       0.89      0.89      0.88      6554


2 Target column offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6554

/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/p

25 Target column shops:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554


26 Target column aid_centers:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6477
           1       0.00      0.00      0.00        77

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554


27 Target column other_infrastructure:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6235
           1       0.00      0.00      0.00       319

    accuracy                           0.95      6554
   macro avg       0.48      0.50

/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# mean of weighted avg precision, recall and f1 
print ('mean of weighted avg precision: {:.2f}'.format(sum(list_precision)/len(list_precision)))
print ('mean of weighted avg recall: {:.2f}'.format(sum(list_recall)/len(list_recall)))
print ('mean of weighted avg f1: {:.2f}'.format(sum(list_f1)/len(list_f1)))

mean of weighted avg precision: 0.94
mean of weighted avg recall: 0.95
mean of weighted avg f1: 0.94


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.